In [1]:
import os
import sys
import json
import torch
from torch import nn
import argparse
import pandas as pd
import numpy as np
from pathlib import Path

# Setup path to src module - find transfer-interference directory
try:
    notebook_path = Path(__file__).parent if '__file__' in globals() else Path.cwd()
except:
    notebook_path = Path.cwd()

# Navigate up to find project root (directory containing 'src' and 'data')
project_root = Path(notebook_path).resolve()
while not (project_root / 'src').exists() or not (project_root / 'data').exists():
    if project_root == project_root.parent:
        # If we can't find it, assume we're in the transfer-interference root
        project_root = Path.cwd()
        break
    project_root = project_root.parent

# Add project root to Python path
sys.path.insert(0, str(project_root))

from src.utils.basic_funcs import set_seed
from src.models import neural_network as net
from src.analysis import ann as ann
from torch.utils.data import Dataset, DataLoader
from src.utils import basic_funcs as basic
import math
import copy
from tqdm.auto import tqdm
from scipy import stats

# Helpers

In [2]:
def numpy_to_python(obj):
    """Convert numpy objects to Python native types for JSON serialization."""
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, dict):
        return {k: numpy_to_python(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [numpy_to_python(item) for item in obj]
    return obj
    
def filter_participant_data(df, participant, task_section):
    """
    Filter participant data by task section.
    """
    return df.loc[
        (df['participant'] == participant) & (df['task_section'] == task_section),
        ['index', 'feature_idx', 'feat_val', 'noisy_feedback_value', 'stimID','test_trial']
    ].reset_index(drop=True)

def adjust_indices(participant_data, offset):
    """
    Adjust the indices of participant data by the specified offset.
    """
    participant_data['index'] -= offset
    return participant_data.reset_index(drop=True)

def create_inputs_matrix(participant_data, n_stim_per_task):
    """
    Create an inputs matrix with one-hot encoded stimulus IDs.
    """
    length = participant_data.shape[0]
    inputs = np.zeros((length, n_stim_per_task * 2))
    for index, row in participant_data.iterrows():
        inputs[index, int(row['stimID'])] = 1
    return inputs

def process_raw_inputs_and_labels(participant_data, n_stim_per_task, task_idx):
    """
    Process raw inputs and labels for a given task.
    """
    unique_inputs = participant_data['stimID'].unique().astype(int)
    raw_inputs = np.full((n_stim_per_task, n_stim_per_task * 2), np.nan, dtype=np.float32)
    raw_labels = np.full((4, n_stim_per_task), np.nan, dtype=np.float32)

    for idx, stim_id in enumerate(unique_inputs):
        feat1 = participant_data.loc[
            (participant_data['stimID'] == stim_id) & (participant_data['feature_idx'] == 0), 'feat_val'
        ].unique()
        feat2 = participant_data.loc[
            (participant_data['stimID'] == stim_id) & (participant_data['feature_idx'] == 1), 'feat_val'
        ].unique()
        raw_labels[0, idx] = np.cos(feat1)[0]
        raw_labels[1, idx] = np.sin(feat1)[0]
        raw_labels[2, idx] = np.cos(feat2)[0]
        raw_labels[3, idx] = np.sin(feat2)[0]

        input_skeleton = np.zeros((n_stim_per_task * 2))
        input_skeleton[stim_id] = 1
        raw_inputs[idx, :] = input_skeleton

    return raw_inputs, raw_labels

def assemble_dataset(participant_data, inputs, label_cos, label_sin):
    """
    Assemble the dataset dictionary for a task.
    """
    return {
        'index': participant_data['index'].values,
        'stim_index': participant_data['stimID'].values,
        'input': inputs,
        'feature_probe': participant_data['feature_idx'].values,
        'test_stim': participant_data['test_trial'].values,
        'label_x': label_cos,
        'label_y': label_sin,
    }
    
def get_datasets(df, participant, task_parameters):
    """
    Main function to get datasets and process raw inputs and labels.
    """
    # Filter data for each task section
    participant_training_A1 = filter_participant_data(df, participant, 'A1')
    participant_training_B = filter_participant_data(df, participant, 'B')
    participant_training_A2 = filter_participant_data(df, participant, 'A2')

    # Adjust indices for B and C
    A_length = len(participant_training_A1)
    B_length = len(participant_training_B)
    participant_training_B = adjust_indices(participant_training_B, A_length)
    participant_training_A2 = adjust_indices(participant_training_A2, A_length + B_length)

    # Create inputs matrices
    A1_inputs = create_inputs_matrix(participant_training_A1, task_parameters['nStim_perTask'])
    B_inputs = create_inputs_matrix(participant_training_B, task_parameters['nStim_perTask'])
    A2_inputs = create_inputs_matrix(participant_training_A2, task_parameters['nStim_perTask'])

    # Process raw inputs and labels
    raw_inputs = np.full((3, task_parameters['nStim_perTask'], task_parameters['nStim_perTask'] * 2), np.nan, dtype=np.float32)
    raw_labels = np.full((3, 4, task_parameters['nStim_perTask']), np.nan, dtype=np.float32)

    raw_inputs[0], raw_labels[0] = process_raw_inputs_and_labels(participant_training_A1, task_parameters['nStim_perTask'], 0)
    raw_inputs[1], raw_labels[1] = process_raw_inputs_and_labels(participant_training_B, task_parameters['nStim_perTask'], 1)
    raw_inputs[2], raw_labels[2] = process_raw_inputs_and_labels(participant_training_A2, task_parameters['nStim_perTask'], 2)

    # Assemble datasets
    dataset_A1 = assemble_dataset(participant_training_A1, A1_inputs, np.cos(participant_training_A1['feat_val'].values), np.sin(participant_training_A1['feat_val'].values))
    dataset_B = assemble_dataset(participant_training_B, B_inputs, np.cos(participant_training_B['feat_val'].values), np.sin(participant_training_B['feat_val'].values))
    dataset_A2 = assemble_dataset(participant_training_A2, A2_inputs, np.cos(participant_training_A2['feat_val'].values), np.sin(participant_training_A2['feat_val'].values))

    return dataset_A1, dataset_B, dataset_A2, raw_inputs, raw_labels

class CreateParticipantDataset(Dataset):
    """PyTorch Dataset for participant data."""
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset['index'])

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample = {key: self.dataset[key][idx] for key in self.dataset}
        if self.transform:
            sample = self.transform(sample)
        return sample

def compute_accuracy(predictions, ground_truth):
    """Compute accuracy between predictions and ground truth in radians."""
    predictions = np.asarray(predictions)
    ground_truth = np.asarray(ground_truth)
    wrapped_difference = basic.wrap_to_pi(predictions - ground_truth)
    normalized_error = np.abs(wrapped_difference) / np.pi
    return 1 - normalized_error

def batch_to_torch(numpy_version):
    """Convert numpy batch to torch tensor."""
    return numpy_version.type(torch.FloatTensor)

In [3]:
def train_participant_schedule(network, trainloader, n_epochs, loss_function, optimizer, do_update, do_test):
    """
    Train the network on x-y coordinates 
    Returns:
        tuple: Various metrics including indexes, inputs, labels, probes, losses, accuracy, predictions, hiddens, embeddings, readouts.
    """
    # Initialize storage lists
    metrics = {
        "indexes": [],
        "losses": [],
        "accuracy": [],
        "predictions": [],
        "hiddens": [],
        "embeddings": [],
        "readouts": [],
        "probes": [],
        "test_stim":[],
        "labels": [],
        "inputs": [],
    }

    for epoch in range(n_epochs):
        for batch_idx, data in enumerate(trainloader):
            # Reset gradients
            optimizer.zero_grad()

            # Extract batch data
            index = data['stim_index']
            input = batch_to_torch(data['input'])
            label_x = batch_to_torch(data['label_x'])
            label_y = batch_to_torch(data['label_y'])
            feature_probe = batch_to_torch(data['feature_probe'])
            test_stim = batch_to_torch(data['test_stim'])
            
                    
            joined_label = torch.cat((label_x.unsqueeze(1), label_y.unsqueeze(1)), dim=1)
            radians_label = math.atan2(label_x, label_y)

            # Forward pass
            out, hid = network(input)

            # Calculate loss based on feature probe
            if feature_probe == 0:
                loss = loss_function(out[:, :2], joined_label)
                pred_rads = math.atan2(out[:, 0].detach().numpy(),out[:, 1].detach().numpy())
                accuracy = compute_accuracy(pred_rads, radians_label)
                
            elif feature_probe == 1:
                loss = loss_function(out[:, 2:4], joined_label)
                pred_rads = math.atan2(out[:, 2].detach().numpy(),out[:, 3].detach().numpy())
                accuracy = compute_accuracy(pred_rads, radians_label)
                
            else:
                raise ValueError("Undefined loss setting for feature_probe.")

            # Update network if required
            if do_update == 1 and do_test==1 and test_stim.numpy() == 0:
              loss.backward()
              optimizer.step()
            elif do_update == 1 and do_test ==0:
              loss.backward()
              optimizer.step()
            elif do_update == 2 and feature_probe == 0:  # In C, only update for feature 0 
              loss.backward()
              optimizer.step()

            # Store metrics
            metrics["indexes"].append(index)
            metrics["inputs"].append(input.numpy())
            metrics["labels"].append(joined_label.numpy())
            metrics["probes"].append(feature_probe.numpy())
            metrics["test_stim"].append(test_stim.numpy())
            metrics["losses"].append(loss.item())
            metrics["accuracy"].append(accuracy)
            metrics["predictions"].append(np.expand_dims(out.detach().numpy(), axis=1))
            metrics["hiddens"].append(hid.detach().numpy())
            metrics["embeddings"].append(network.in_hid.weight.detach().numpy())
            metrics["readouts"].append(network.hid_out.weight.detach().numpy())

    # Convert lists to arrays where applicable
    metrics = {key: np.squeeze(value) for key, value in metrics.items()}
    
    return (
        metrics["indexes"],
        metrics["inputs"],
        metrics["labels"],
        metrics["probes"],
        metrics["test_stim"],
        metrics["losses"],
        metrics["accuracy"],
        metrics["predictions"],
        metrics["hiddens"],
        metrics["embeddings"],
        metrics["readouts"],
    )

# Data

In [4]:
# Set random seed
set_seed(2024)
condition_name = "rich_50" # Condition to run (e.g., rich_10, rich_50, rich_200)
base_folder='./'


In [5]:
# Setup paths
data_folder = os.path.join(base_folder, 'data')
config_path = os.path.join(base_folder, 'src', 'models', 'ann_experiments.json')

In [6]:
# Load settings and find specified condition
settings = json.load(open(config_path, 'r'))

condition = next((c for c in settings['conditions'] if c['name'] == condition_name), None)
if not condition:
    raise ValueError(f"Condition '{condition_name}' not found in settings")

condition

{'name': 'rich_50', 'gamma': 0.001, 'dim_hidden': 50}

In [7]:
# Load participant data
df = pd.read_csv(os.path.join(data_folder, 'participants', 'trial_df.csv'))

df.loc[df['task_section']=='B','test_trial']=0

df = df.loc[(df['task_section']=='A1') | 
                 (df['task_section']=='B') | 
                 (df['task_section']=='A2'), :] # remove debrief trials from analysis

participants = df['participant'].unique()

df


,participant,index,task_section,feature_idx,stimID,taskID,feat_val,noisy_feedback_value,resp_reactiontime,dial_resp,...,block,regime,accuracy,study,A_rule,B_rule,rule_applied,test_stim_B,test_stim_A,test_trial
0,study1_same_sub9,0,A1,0.0,2.0,0.0,0.900836,0.910773,12.212,4.886953,...,0,study1_same,0.268820,1,-1.901668,-1.901668,NaN,5.0,9.0,0
1,study1_same_sub9,1,A1,1.0,2.0,0.0,5.282353,5.264300,8.746,3.202714,...,0,study1_same,0.338030,1,-1.901668,-1.901668,2.291941,5.0,9.0,0
2,study1_same_sub9,2,A1,0.0,0.0,0.0,5.849861,5.946899,10.087,3.227958,...,0,study1_same,0.165422,1,-1.901668,-1.901668,NaN,5.0,9.0,0
3,study1_same_sub9,3,A1,1.0,0.0,0.0,3.948193,4.032603,8.283,2.462548,...,0,study1_same,0.527105,1,-1.901668,-1.901668,2.798835,5.0,9.0,0
4,study1_same_sub9,4,A1,0.0,11.0,0.0,4.051682,4.148304,4.658,3.951195,...,0,study1_same,0.968014,1,-1.901668,-1.901668,NaN,5.0,9.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117357,study1_far_sub57,355,A2,1.0,0.0,0.0,1.375241,NaN,3.314,4.466989,...,29,study1_far,0.015866,1,1.788678,-1.352915,-1.351399,7.0,0.0,1
117358,study1_far_sub57,356,A2,0.0,9.0,0.0,4.865064,4.963278,2.577,2.791894,...,29,study1_far,0.340090,1,1.788678,-1.352915,NaN,7.0,0.0,0
117359,study1_far_sub57,357,A2,1.0,9.0,0.0,0.370556,NaN,1.249,3.222413,...,29,study1_far,0.092226,1,1.788678,-1.352915,-1.740865,7.0,0.0,0
117360,study1_far_sub57,358,A2,0.0,5.0,0.0,5.273465,5.240881,2.183,0.693492,...,29,study1_far,0.457851,1,1.788678,-1.352915,NaN,7.0,0.0,0


In [8]:
participants.shape

(305,)

# Setup

In [9]:
# Setup parameters
task_parameters = {
        "nStim_perTask": 6,
        "schedules": ['same', 'near', 'far'],
        "schedule_names": ['same rule', 'near rule', 'far rule']
    }

# Network parameters
dim_input = task_parameters['nStim_perTask'] * 2
dim_hidden = condition['dim_hidden']
dim_output = 4  # 2 dimensions for each feature
network_params = [dim_input, dim_hidden, dim_output]


# Training parameters - convert to list format expected by neural_network.py
training_params = [
    participants,  # participants list
    settings['n_phase'],  # n_phase
    settings['n_epochs'],  # n_epochs
    settings['n_epochs'] * (task_parameters['nStim_perTask']*2) * 10,  # n_train_trials
    settings['shuffle'],  # shuffle
    settings['batch_size'],  # batch_size
    condition['gamma'],  # gamma
    settings['learning_rate'],  # learning rate
]


In [10]:
# Setup simulation folder if not existent yet
sim_folder = os.path.join(data_folder, 'any_network', condition_name)
os.makedirs(sim_folder, exist_ok=True)

In [11]:
# Save settings of the run
settings_to_save = {
    "condition": condition,
    "training_params": {
        "participants": ann.numpy_to_python(participants),
        "n_phase": settings['n_phase'],
        "n_epochs": settings['n_epochs'],
        "n_train_trials": settings['n_epochs'] * (task_parameters['nStim_perTask']*2) * 10,
        "shuffle": settings['shuffle'],
        "batch_size": settings['batch_size'],
        "gamma": condition['gamma'],
        "lr": settings['learning_rate'],
    },
    "network_params": network_params,
    "task_parameters": task_parameters
}

# Convert numpy arrays to Python native types before saving
settings_to_save = ann.numpy_to_python(settings_to_save)
with open(os.path.join(sim_folder, 'settings.json'), 'w') as f:
    json.dump(settings_to_save, f, indent=4)

In [12]:
# Unpack parameters
dim_input, dim_hidden, dim_output = network_params
participants, n_phase, n_epochs, n_train_trials, shuffle, batch_size, gamma, lr = training_params

# add these params
do_test = 1
dosave=1

results = []

## Network

In [13]:
class simpleLinearNet(nn.Module):
    """A simple linear neural network with one hidden layer.
    
    Architecture:
    input -> hidden layer -> output
    All layers are fully connected with no bias terms.
    """
    def __init__(self, dim_input, dim_hidden, dim_output):
        super(simpleLinearNet, self).__init__()
        self.in_hid = nn.Linear(dim_input, dim_hidden, bias=False)
        self.hid_out = nn.Linear(dim_hidden, dim_output, bias=False)
        
    def forward(self, x):
        """Forward pass through the network."""
        hid = self.in_hid(x)
        out = self.hid_out(hid)
        return out, hid

def ex_initializer_(model, gamma=1e-3,mean=0.0):
    """
    In-place Re-initialization of weights

    Args:
        model: torch.nn.Module
        PyTorch neural net model
        
        gamma: float
        Initialization scale

    Returns:
        Nothing
    """
    for name, param in model.named_parameters():
        if "weight" in name:  
            n_out, n_in = param.shape
                
            if "hid_out" in name:  # Output layer weights
                std = 1e-3
            else:  # Hidden layer weights
                std = gamma
                
            nn.init.normal_(param, mean=mean, std=std)

def ordered_sweep(network, ranked_inputs):
    """Run network on ordered inputs for interpretable results."""
    preds, hids = network(ranked_inputs)
    return preds.detach().numpy().copy(), hids.detach().numpy().copy()

# Train

In [14]:
# for each participant

participant_results = {}

for idx_p, participant in tqdm(enumerate(participants[0:10])): # test for first participant only
    print(f'Starting participant {participant}')

    # Get participant data
    dataset_A1, dataset_B, dataset_A2, raw_inputs, raw_labels = basic.get_datasets(df, participant, task_parameters)
    
    # Order inputs by feature
    A_inputs = raw_inputs[0]
    B_inputs = raw_inputs[1] 
    A_labels_feat1 = raw_labels[0, 0:2].T
    B_labels_feat1 = raw_labels[1, 0:2].T
    ordered_indices_A = basic.get_clockwise_order(A_labels_feat1)
    ordered_indices_B = basic.get_clockwise_order(B_labels_feat1)
    ordered_inputs = np.concatenate((A_inputs[ordered_indices_A], B_inputs[ordered_indices_B]), axis=0)

    # Create data loaders
    trainloader_A1 = DataLoader(CreateParticipantDataset(dataset_A1), batch_size=batch_size, shuffle=shuffle)
    trainloader_B = DataLoader(CreateParticipantDataset(dataset_B), batch_size=batch_size, shuffle=shuffle)
    trainloader_A2 = DataLoader(CreateParticipantDataset(dataset_A2), batch_size=batch_size, shuffle=shuffle)


    # Run a complete learning cycle
    """
    Runs a complete learning cycle:
    A: n_epochs of training on task A stimuli
    B: n_epochs of training on task B stimuli
    """
    n_train_trials = n_epochs * dim_input * 10
    n_phase = 3  # A, B, A

    # Preallocate results matrices
    results = {
        "indexes": np.full((n_phase, n_train_trials), np.nan, dtype=np.float32),
        "inputs": np.full((n_phase, n_train_trials, dim_input), np.nan, dtype=np.float32),
        "labels": np.full((n_phase, n_train_trials, 2), np.nan, dtype=np.float32),
        "test_stim": np.full((n_phase, n_train_trials), np.nan, dtype=np.float32),
        "probes": np.full((n_phase, n_train_trials), np.nan, dtype=np.float32),
        "losses": np.full((n_phase, n_train_trials), np.nan, dtype=np.float32),
        "accuracy": np.full((n_phase, n_train_trials), np.nan, dtype=np.float32),
        "predictions": np.full((n_phase, n_train_trials, dim_output), np.nan, dtype=np.float32),
        "hiddens": np.full((n_phase, n_train_trials, dim_hidden), np.nan, dtype=np.float32),
        "embeddings": np.full((n_phase, n_train_trials, dim_hidden, dim_input), np.nan, dtype=np.float32),
        "readouts": np.full((n_phase, n_train_trials, dim_output, dim_hidden), np.nan, dtype=np.float32),
    }

    # Define the network
    network = simpleLinearNet(dim_input, dim_hidden, dim_output)

    # Initialize weights
    ex_initializer_(network, gamma)

    optimizer = torch.optim.SGD(network.parameters(), lr=lr)
    loss_function = nn.MSELoss()

    # Initial pass of the network
    initial_preds, initial_hiddens = ordered_sweep(network, torch.from_numpy(ordered_inputs).float())

    results["preds_pre_training"] = initial_preds
    results["hiddens_pre_training"] = initial_hiddens

    
    # Training Phases
    phases = [
        (0, trainloader_A1, 1),
        (1, trainloader_B, 1),
        (2, trainloader_A2, 2),
    ]

    # Phase A1
    phase = 0
    loader =  trainloader_A1
    do_update = 1 # Controls how updates are applied (0 = no update, 1 = standard, 2 = conditional on feature_probe).

    (
        results["indexes"][phase, :],
        results["inputs"][phase, :, :],
        results["labels"][phase, :, :],
        results["probes"][phase, :],
        results["test_stim"][phase, :],
        results["losses"][phase, :],
        results["accuracy"][phase, :],
        results["predictions"][phase, :, :],
        results["hiddens"][phase, :, :],
        results["embeddings"][phase, :, :, :],
        results["readouts"][phase, :, :, :],
    ) = train_participant_schedule(
        network, loader, n_epochs, loss_function, optimizer, do_update, do_test
    )

    # Post-phase ordered sweep
    post_preds, post_hiddens = ordered_sweep(network, torch.from_numpy(ordered_inputs).float())
    results[f"preds_post_phase_{phase}"] = post_preds
    results[f"hiddens_post_phase_{phase}"] = post_hiddens

    phase = 1
    loader =  trainloader_B
    do_update = 1 # Controls how updates are applied (0 = no update, 1 = standard, 2 = conditional on feature_probe).

    (
        results["indexes"][phase, :],
        results["inputs"][phase, :, :],
        results["labels"][phase, :, :],
        results["probes"][phase, :],
        results["test_stim"][phase, :],
        results["losses"][phase, :],
        results["accuracy"][phase, :],
        results["predictions"][phase, :, :],
        results["hiddens"][phase, :, :],
        results["embeddings"][phase, :, :, :],
        results["readouts"][phase, :, :, :],
    ) = train_participant_schedule(
        network, loader, n_epochs, loss_function, optimizer, do_update, do_test
    )


    # Post-phase ordered sweep
    post_preds, post_hiddens = ordered_sweep(network, torch.from_numpy(ordered_inputs).float())
    results[f"preds_post_phase_{phase}"] = post_preds
    results[f"hiddens_post_phase_{phase}"] = post_hiddens


    phase = 2
    loader =  trainloader_A2
    do_update = 2  # Controls how updates are applied (0 = no update, 1 = standard, 2 = conditional on feature_probe).

    (
        results["indexes"][phase, :],
        results["inputs"][phase, :, :],
        results["labels"][phase, :, :],
        results["probes"][phase, :],
        results["test_stim"][phase, :],
        results["losses"][phase, :],
        results["accuracy"][phase, :],
        results["predictions"][phase, :, :],
        results["hiddens"][phase, :, :],
        results["embeddings"][phase, :, :, :],
        results["readouts"][phase, :, :, :],
    ) = train_participant_schedule(
        network, loader, n_epochs, loss_function, optimizer, do_update, do_test
    )

    # Post-phase ordered sweep
    post_preds, post_hiddens = ordered_sweep(network, torch.from_numpy(ordered_inputs).float())
    results[f"preds_post_phase_{phase}"] = post_preds
    results[f"hiddens_post_phase_{phase}"] = post_hiddens

    results['participant'] = participant


    participant_results[idx_p] = results

    """
    # Save results if requested
    if dosave:
        file_path = f"{sim_folder}/sim_{participant}.npz"
        np.savez_compressed(file_path, **results)

    # Cleanup
    del participant_results

    """


0it [00:00, ?it/s]

Starting participant study1_same_sub9


/tmp/ipykernel_2791137/1797680934.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_rads = math.atan2(out[:, 0].detach().numpy(),out[:, 1].detach().numpy())
/tmp/ipykernel_2791137/1797680934.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_rads = math.atan2(out[:, 2].detach().numpy(),out[:, 3].detach().numpy())


Starting participant study2_near_sub43


/tmp/ipykernel_2791137/1797680934.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_rads = math.atan2(out[:, 0].detach().numpy(),out[:, 1].detach().numpy())
/tmp/ipykernel_2791137/1797680934.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_rads = math.atan2(out[:, 2].detach().numpy(),out[:, 3].detach().numpy())


...

/tmp/ipykernel_2791137/1797680934.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_rads = math.atan2(out[:, 0].detach().numpy(),out[:, 1].detach().numpy())
/tmp/ipykernel_2791137/1797680934.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_rads = math.atan2(out[:, 2].detach().numpy(),out[:, 3].detach().numpy())


Starting participant study2_same_sub20


/tmp/ipykernel_2791137/1797680934.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_rads = math.atan2(out[:, 0].detach().numpy(),out[:, 1].detach().numpy())
/tmp/ipykernel_2791137/1797680934.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_rads = math.atan2(out[:, 2].detach().numpy(),out[:, 3].detach().numpy())


Starting participant study1_same_sub56


/tmp/ipykernel_2791137/1797680934.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_rads = math.atan2(out[:, 0].detach().numpy(),out[:, 1].detach().numpy())
/tmp/ipykernel_2791137/1797680934.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_rads = math.atan2(out[:, 2].detach().numpy(),out[:, 3].detach().numpy())


## Results

In [15]:
participant_results[7]

{'indexes': array([[ 7.,  7.,  2., ..., 11.,  7.,  7.],
        [ 3.,  3.,  8., ...,  8.,  5.,  5.],
        [ 7.,  7., 11., ..., 10.,  9.,  9.]],
       shape=(3, 12000), dtype=float32),
 'inputs': array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 1.],
         ...,
         [0., 0., 0., ..., 0., 1., 0.],
         [0., 0., 0., ..., 1., 0., 0.],
         [0., 0., 0., ..., 1., 0., 0.]]],
       shape=(3, 12000, 12), dty

## save and clean

In [16]:
# Save results if requested
if dosave:
    file_path = f"{sim_folder}/sim_{participant}.npz"
    np.savez_compressed(file_path, **participant_results)

# Cleanup
del participant_results

TypeError: keywords must be strings

In [ ]:
ann_data = ann.load_ann_data(project_root / 'data/simulations/rich_50')